In [1]:
import json
import requests
import os

# os.environ['NO_PROXY'] = '127.0.0.1'
url = "http://127.0.0.1:5000"

## Get info


In [8]:
response = requests.post(url + "/routeGetDatasetsInfo", {})
dataset = 'wesad_dimensional'
json.loads(response.content)

{'loadedDatasetsIds': ['wesad_dimensional', 'wesad_categorical'],
 'localDatasetsIds': ['case', 'wesad_dimensional', 'wesad_categorical']}

## Load local dataset

In [6]:
response = requests.post(url + "/loadLocalDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}

## Remove dataset

In [17]:
response = requests.post(url + "/removeDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'error'}

## Get Dataset info

In [9]:
response = requests.post(url + "/getDatasetInfo",data = {'datasetId':dataset})

info = json.loads(response.content)
info

{'categoricalLabels': {'Dominant hand': ['left', 'right'],
  'Gender': ['female', 'male']},
 'dimensions': {'Arousal': 'arousal', 'Valence': 'valence'},
 'globalEmotionMax': {'Arousal': 9, 'Valence': 9},
 'globalEmotionMin': {'Arousal': 0, 'Valence': 0},
 'ids': ['s10',
  's11',
  's13',
  's14',
  's15',
  's16',
  's17',
  's2',
  's3',
  's4',
  's5',
  's6',
  's7',
  's8',
  's9'],
 'instanceLen': 15,
 'isDated': False,
 'numericalLabels': ['Age', 'Height (cm)', 'Weight (kg)'],
 'seriesLabels': ['Valence', 'Arousal'],
 'temporalLen': 5,
 'type': 'dimensional',
 'variablesLen': 32}

## Get Time Series

In [69]:
response = requests.post(url + "/getMTSeries",data = {'datasetId':dataset, 'begin': 0, 'end': 4, 'ids': json.dumps(['1', '2', '5'])})
json.loads(response.content)

{'1': {'categoricalFeatures': [],
  'categoricalLabels': [],
  'index': ['2021-02-10 01:55:19',
   '2021-02-10 01:55:19.050000',
   '2021-02-10 01:55:19.100000',
   '2021-02-10 01:55:19.150000'],
  'metadata': {'id': '1'},
  'numericalFeatures': [],
  'numericalLabels': [],
  'temporalVariables': {'Arousal': [5.0, 5.0, 5.0, 5.0],
   'Valence': [5.0, 5.0, 5.0, 5.0],
   'Video': [10.0, 10.0, 10.0, 10.0]}},
 '2': {'categoricalFeatures': [],
  'categoricalLabels': [],
  'index': ['2021-02-10 01:55:19',
   '2021-02-10 01:55:19.050000',
   '2021-02-10 01:55:19.100000',
   '2021-02-10 01:55:19.150000'],
  'metadata': {'id': '2'},
  'numericalFeatures': [],
  'numericalLabels': [],
  'temporalVariables': {'Arousal': [5.0, 5.0, 5.0, 5.0],
   'Valence': [5.0, 5.0, 5.0, 5.0],
   'Video': [10.0, 10.0, 10.0, 10.0]}},
 '5': {'categoricalFeatures': [],
  'categoricalLabels': [],
  'index': ['2021-02-10 01:55:19',
   '2021-02-10 01:55:19.050000',
   '2021-02-10 01:55:19.100000',
   '2021-02-10 01:55:1

## Get projection


In [49]:
alphas = {label: 1 for label in info["seriesLabels"]}
D_k = {}
coords = {}

In [50]:

response = requests.post(url + "/getDatasetProjection",data = {'datasetId':dataset, 'alphas': json.dumps(alphas), 'begin': 0, 'end':40, 
                                                               'D_k': json.dumps(D_k), 'oldCoords': json.dumps(coords)})
data = json.loads(response.content)
coords = data["coords"]
D_k = data["D_k"]
coords

{'1': [10.331422178159485, -7.822703821703016],
 '2': [9.149584635408338, 1.78141223125199],
 '3': [-10.179948481847532, -1.4284377541435411],
 '4': [-3.7285033656149005, -10.076584413672705],
 '5': [-7.418150277018119, 9.925811489772995],
 '6': [1.845595310912727, 7.620502268494276]}

## Clustering

In [38]:
response = requests.post(url + "/doClustering",data = {'datasetId':dataset, 'coords': json.dumps(coords), 'k': 2})
clusters = json.loads(response.content)
clusters

{'0': ['1', '2', '4', '5'], '1': ['3', '6']}

## Rankings

In [39]:
response = requests.post(url + "/getFishersDiscriminantRanking",data = {'datasetId':dataset, 'D_k': json.dumps(D_k), 'blueCluster': json.dumps(clusters['0']), 'redCluster': json.dumps(clusters['1']) })
j_s = json.loads(response.content)
j_s

{'Arousal': 0.051812307045928346, 'Valence': 3.415525686489257}

## Temporal summary

In [41]:
response = requests.post(url + "/getTemporalSummary",data = {'datasetId':dataset})
json.loads(response.content)["mean"]


{'Arousal': [5.559671133137543,
  5.172665973241965,
  5.335087453091859,
  5.316978870941426,
  4.931075318159569,
  4.985956457007668],
 'Valence': [5.22886739272312,
  4.902533223201175,
  5.231475383423071,
  4.878587820199053,
  5.036623715124817,
  5.253636441507587]}

## Downsample

In [71]:
response = requests.post(url + "/downsampleData",data = {'datasetId':dataset, 'rule': 'S'})
json.loads(response.content)

{'state': 'success'}

## Reset dataset

In [68]:
response = requests.post(url + "/resetDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}